# Bond Equity strategies

Objective: Build a strategy that outperforms the traditional 60/40 portfolio

Bond: [TLT](https://www.ishares.com/us/products/239454/ishares-20-year-treasury-bond-etf)\
Equity: [SPY](https://www.ssga.com/us/en/intermediary/etfs/spdr-sp-500-etf-trust-spy)

**Notes** 
* Due to my employer's personal dealing policy, I am restricted to trading only once every 30 days, so my strategy uses a rebalancing period of 31 days (additional day to account for approval process).


# Import Packages

In [1]:
import openbb as obb
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sklearn as sk
from typing import Union, List, Tuple, Dict, Callable
import datetime as dt
import calendar
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "jupyterlab"
import math
import os
import arch
from Backtest import Backtest 

In [2]:
import scipy as sci

## Helpers

In [3]:
def show_full_df(data: pd.DataFrame) -> None:
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
        display(data)

# Benchmark - Buy SPY

In [4]:
# Import data from OpenBB
start: dt.date = dt.date(year= 2000,month= 1, day=1)
end: dt.date = dt.date(year= 2025,month= 1, day=1)

SPY_raw: pd.DataFrame = obb.obb.equity.price.historical(symbol= "SPY", start_date= start, end_date = end).to_df()
SPY: pd.DataFrame = SPY_raw[["close","volume"]].rename({"close":"SPY"}, axis = 1)

In [5]:
def buy_and_hold_func(tick: dt.date,securities: List = ["SPY"], data= SPY, lookback = 30) -> bool:
    return (1,{"Add_data":{}})

In [6]:
buy_and_hold: Backtest = Backtest(name= "Buy and hold", data = SPY)

In [7]:
buy_and_hold.securities = ["SPY"]
buy_and_hold.assign_rebal_attr(day_of_rebal = 1, interval = "quarter")
buy_and_hold.assign_signals(trading_signal= buy_and_hold_func)
buy_and_hold.lookback_window = 15

In [8]:
benchmark: pd.DataFrame = buy_and_hold.calculate()

In [9]:
buy_and_hold.plot()

In [10]:
buy_and_hold.stats

,Returns,Volatility,Sharpe
Results,0.085997,0.19379,0.443765


In [11]:
# Function for strategy comparison 
def outperformance(strategy: Backtest, benchmark: Backtest = buy_and_hold, measure = "Sharpe"): 
    try:
        flag: int = 1
        if measure == "Volatility": 
            flag: int = -1 
        out_performance = float((benchmark.stats[measure] - strategy.stats[measure]).iloc[0])*-1 # I get a strange FutureWarning error if I do this the other way around 
        print(f"Outperformance ({measure}): %.3f" %(out_performance*flag))
    except:
        print("Make sure Benchmark strategy has been run")
    

# Import data

In [12]:
TLT_raw: pd.DataFrame = obb.obb.equity.price.historical(symbol= "TLT", start_date= start, end_date = end).to_df()

In [13]:
TLT_SPY = SPY.merge(TLT_raw.rename({"close":"TLT"}, axis = 1).TLT, left_index=True, right_index=True)
TLT_SPY.drop("volume", axis = 1, inplace = True)

# Visualisation and initial analysis

In [14]:
fig1 = px.line(TLT_SPY, x = TLT_SPY.index, y = ["SPY", "TLT"])
fig1.show(renderer= "iframe")

In [15]:
# Calculate log returns for TLT and SPY
TLT_SPY_logret = (np.log(TLT_SPY) - np.log(TLT_SPY.shift(1))).dropna(how = "any")

In [16]:
# Calculate correlation of TLT and SPY log returns 
TLT_SPY_logret.corr()

,SPY,TLT
SPY,1.00000,-0.32641
TLT,-0.32641,1.00000


In [17]:
# Plot scatter of log returns 
fig2 = px.scatter(TLT_SPY_logret, x = "TLT", y = "SPY")
fig2.show(renderer= "iframe")

In [18]:
# Simple regression model of log returns 
reg0_model = sm.regression.linear_model.OLS(TLT_SPY_logret.SPY, TLT_SPY_logret.TLT)
reg0_results = reg0_model.fit()
reg0_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    SPY   R-squared (uncentered):                   0.106
Model:                            OLS   Adj. R-squared (uncentered):              0.106
Method:                 Least Squares   F-statistic:                              672.2
Date:                Sun, 30 Mar 2025   Prob (F-statistic):                   4.08e-140
Time:                        15:11:08   Log-Likelihood:                          17300.
No. Observations:                5644   AIC:                                 -3.460e+04
Df Residuals:                    5643   BIC:                                 -3.459e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TLT           -0.4260      0.016    -25.926      0.000      -0.458      -0.394
==============================================================================
Omnibus:                     1152.497   Durbin-Watson:                   2.147
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            35251.501
Skew:                          -0.225   Prob(JB):                         0.00
Kurtosis:                      15.235   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Comment** 
* Whilst the T-statistic is very high, the R<sup>2</sup> is very low (would expect this given how naive it is)

In [19]:
# Rolling correlation between log returns
rolling_corr = TLT_SPY_logret[["SPY","TLT"]].rolling(60).corr().unstack().iloc[:,1].dropna()
rolling_corr.name = "SPY_TLT"

fig3 = px.line(rolling_corr, x = rolling_corr.index, y = "SPY_TLT")
fig3.show(renderer= "iframe")

# Strategies

In [20]:
# Split data in half for training 
train_TLT_SPY = TLT_SPY.iloc[:int(len(TLT_SPY)/2)]

## Simple 60/40 portfolio

In [21]:
def TLT_SPY_0(tick, securities: List = ["SPY","TLT"], data = train_TLT_SPY, lookback = 180) -> Tuple[int, Dict]:
    index_loc_tic: int = data.index.get_loc(tick)
    lb_window = data.iloc[index_loc_tic-lookback:index_loc_tic]

    sig: tuple = (0.6,0.4)
    

    return (sig, {"add_data":{}})

In [22]:
TLT_SPY0: Backtest = Backtest(name= "Basic 60/40", data = train_TLT_SPY)

In [23]:
TLT_SPY0.securities = ["SPY", "TLT"]
TLT_SPY0.assign_rebal_attr(fixed_ticks=31)
TLT_SPY0.assign_signals(trading_signal= TLT_SPY_0)
TLT_SPY0.lookback_window = 60

In [24]:
TLT_SPY0.calculate()

,SPY,TLT,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs,Add_data
date,,,,,,,,,
2002-10-23,90.199997,84.330002,100.000000,0.665188,0.474327,0.6,0.4,NaN,[{'add_data': {}}]
2002-10-24,88.360001,85.070000,99.127057,0.665188,0.474327,0.6,0.4,0.0,NaN
2002-10-25,90.199997,85.379997,100.498041,0.665188,0.474327,0.6,0.4,0.0,NaN
2002-10-28,89.610001,85.260002,100.048666,0.665188,0.474327,0.6,0.4,0.0,NaN
2002-10-29,88.570000,86.370003,99.883372,0.665188,0.474327,0.6,0.4,0.0,NaN
...,...,...,...,...,...,...,...,...,...
2013-10-07,167.429993,106.139999,172.603651,0.617619,0.652131,0.6,0.4,0.0,NaN
2013-10-08,165.479996,106.169998,171.418859,0.617619,0.652131,0.6,0.4,0.0,NaN
2013-10-09,165.600006,105.320000,170.938669,0.621533,0.645828,0.6,0.4,0.0,"[{'add_data': {}, 'rebal_data': (True, True, N..."


In [25]:
TLT_SPY0.plot()

In [26]:
TLT_SPY0.stats

,Returns,Volatility,Sharpe
Results,0.075881,0.108423,0.699859


In [27]:
outperformance(TLT_SPY0)

Outperformance (Sharpe): 0.256


In [28]:
outperformance(TLT_SPY0, measure = "Returns")

Outperformance (Returns): -0.010


In [29]:
outperformance(TLT_SPY0, measure = "Volatility")

Outperformance (Volatility): 0.085


**Comments**
* Large drawdown in 2008 down to below initial investment amount
    * I suspect this is driven by equity underperformance, coupled with positive correlation between SPY and TLT in that period
* Adding TLT has certainly reduced annualised returns but reduced volatility by a greater amount

## Dynamic weights - Simple regression 

Idea: Use a simple regression model to inform the weights 

In [30]:
sm.regression.linear_model.RegressionModel

statsmodels.regression.linear_model.RegressionModel

In [31]:
def TLT_SPY_1(tick, securities: List = ["SPY","TLT"], data = train_TLT_SPY, lookback = 180) -> Tuple[int, Dict]:
    index_loc_tic: int = data.index.get_loc(tick)
    lb_window = data.iloc[index_loc_tic-lookback:index_loc_tic]

    lb_log_ret = (np.log(lb_window)-np.log(lb_window.shift(1))).dropna(how = "any")

    reg_model = sm.regression.linear_model.OLS(lb_log_ret.SPY, lb_log_ret.TLT)
    reg_results = reg_model.fit()

    TLT_coef: float = reg_results.params.iloc[0]

    SPY_w = 1/(1+abs(TLT_coef))
    TLT_w = 1 - SPY_w

    sig: tuple = (SPY_w,TLT_w)

    return (sig, {"add_data":{"beta": TLT_coef}})

In [32]:
TLT_SPY1: Backtest = Backtest(name= "SPY TLT dynamic weights", data = train_TLT_SPY)

TLT_SPY1.securities = ["SPY", "TLT"]
TLT_SPY1.assign_rebal_attr(fixed_ticks=31)
TLT_SPY1.assign_signals(trading_signal= TLT_SPY_1)
TLT_SPY1.lookback_window = 60

In [33]:
TLT_SPY1.calculate()

,SPY,TLT,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs,Add_data
date,,,,,,,,,
2002-10-23,90.199997,84.330002,100.000000,0.406302,0.751234,0.366484,0.633516,NaN,[{'add_data': {'beta': -1.7286291645719383}}]
2002-10-24,88.360001,85.070000,99.808317,0.406302,0.751234,0.366484,0.633516,0.0,NaN
2002-10-25,90.199997,85.379997,100.788792,0.406302,0.751234,0.366484,0.633516,0.0,NaN
2002-10-28,89.610001,85.260002,100.458931,0.406302,0.751234,0.366484,0.633516,0.0,NaN
2002-10-29,88.570000,86.370003,100.870247,0.406302,0.751234,0.366484,0.633516,0.0,NaN
...,...,...,...,...,...,...,...,...,...
2013-10-07,167.429993,106.139999,216.137248,1.200765,0.141439,0.930777,0.069223,0.0,NaN
2013-10-08,165.479996,106.169998,213.800003,1.200765,0.141439,0.930777,0.069223,0.0,NaN
2013-10-09,165.600006,105.320000,213.823885,1.232830,0.092223,0.954203,0.045797,0.0,"[{'add_data': {'beta': -0.047994570132664394},..."


In [34]:
TLT_SPY1.output.describe()

,SPY,TLT,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs
count,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2761.0
mean,122.957346,96.676846,150.588186,0.840132,0.466456,0.694046,0.305954,0.0
std,20.538104,12.043123,31.747038,0.179354,0.246811,0.168003,0.168003,0.0
min,68.110001,80.650002,98.577690,0.406302,0.019623,0.360638,0.014229,0.0
25%,110.202497,88.412502,126.983783,0.719588,0.219667,0.568977,0.157821,0.0
50%,123.155003,92.115002,146.033318,0.861508,0.507014,0.687659,0.312341,0.0
75%,137.047501,101.902502,172.232852,0.967784,0.690264,0.842179,0.431023,0.0
max,173.050003,132.160004,222.722903,1.232830,0.851442,0.985771,0.639362,0.0


In [35]:
TLT_SPY1.plot()

In [36]:
TLT_SPY1.stats

,Returns,Volatility,Sharpe
Results,0.109538,0.104603,1.047181


In [37]:
outperformance(TLT_SPY1)

Outperformance (Sharpe): 0.603


In [38]:
outperformance(TLT_SPY1, benchmark = TLT_SPY0)

Outperformance (Sharpe): 0.347


In [39]:
outperformance(TLT_SPY1, measure = "Returns")

Outperformance (Returns): 0.024


**Comments**
* Significant improvement in Sharpe vs 60/40
* Still seeing sluggish performance between 2007 and 2009
* Interestingly, returns went up vs Buy-and-hold strategy

## Controlling for Volatility - adding VIX

Idea: Volatility is a key variable that dictates the relationship between bonds and stocks - stocks and bonds move together (usually down) during periods of high volatility. The most naive way to do this is to incorporate VIX into the simple regression approach above. 

In [40]:
# Import data from OpenBB
VIX_raw: pd.DataFrame = obb.obb.equity.price.historical(symbol= "^VIX", start_date= start, end_date= end).to_df()
VIX = VIX_raw[["close"]].rename({"close":"VIX"}, axis = 1)

In [41]:
# TSV for "TLT, SPY, VIX"
TSV = TLT_SPY.merge(VIX, left_index= True, right_index = True)

In [42]:
# Calculate log returns
TSV_log_ret = (np.log(TSV) - np.log(TSV.shift(1))).dropna(how = "any")

In [43]:
reg2_model = sm.regression.linear_model.OLS(TSV_log_ret.SPY, TSV_log_ret[["TLT","VIX"]])
reg2_results = reg2_model.fit()
reg2_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    SPY   R-squared (uncentered):                   0.548
Model:                            OLS   Adj. R-squared (uncentered):              0.548
Method:                 Least Squares   F-statistic:                              3422.
Date:                Sun, 30 Mar 2025   Prob (F-statistic):                        0.00
Time:                        15:12:16   Log-Likelihood:                          19224.
No. Observations:                5644   AIC:                                 -3.844e+04
Df Residuals:                    5642   BIC:                                 -3.843e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TLT           -0.2033      0.012    -16.848      0.000      -0.227      -0.180
VIX           -0.1128      0.002    -74.260      0.000      -0.116      -0.110
==============================================================================
Omnibus:                     1516.638   Durbin-Watson:                   2.147
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            92246.409
Skew:                           0.393   Prob(JB):                         0.00
Kurtosis:                      22.790   Cond. No.                         8.21
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Comment
* The stand-out observation here is the improvement in R<sup>2</sup>

In [44]:
fig4 = px.scatter_3d(TSV_log_ret, x = "SPY", y = "VIX", z = "TLT").update_traces(marker = dict(size = 1))
fig4.show(renderer= "iframe")

**Comments**
* Clear negative relationship between VIX log returns and SPY
* Conscious that the absolute level of the VIX is important but changes in the VIX are also a key thing to consider

In [45]:
# Split data in half for training
train_TSV = TSV.iloc[:int(len(TSV)/2)]

In [46]:
def TLT_SPY_2(tick, securities: List = ["SPY","TLT"], data = train_TSV, lookback = 180) -> Tuple[int, Dict]:
    index_loc_tic: int = data.index.get_loc(tick)
    lb_window = data.iloc[index_loc_tic-lookback:index_loc_tic]

    lb_log_ret = (np.log(lb_window)-np.log(lb_window.shift(1))).dropna(how = "any")

    reg_model = sm.regression.linear_model.OLS(lb_log_ret.SPY, lb_log_ret[["TLT","VIX"]]) 
    reg_results = reg_model.fit()

    TLT_coef: float = reg_results.params.loc["TLT"]

    SPY_w = 1/(1+abs(TLT_coef))
    TLT_w = 1 - SPY_w

    sig: tuple = (SPY_w,TLT_w)

    return (sig, {"add_data":{"beta": TLT_coef}})

In [47]:
TLT_SPY2: Backtest = Backtest(name= "SPY TLT VIX dynamic weights", data = train_TSV)
TLT_SPY2.securities = ["SPY", "TLT"]
TLT_SPY2.assign_rebal_attr(fixed_ticks=31)
TLT_SPY2.assign_signals(trading_signal= TLT_SPY_2)
TLT_SPY2.lookback_window = 60

In [48]:
TLT_SPY2.calculate()

,SPY,TLT,VIX,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs,Add_data
date,,,,,,,,,,
2002-10-23,90.199997,84.330002,33.200001,100.000000,0.673078,0.465888,0.607117,0.392883,NaN,[{'add_data': {'beta': -0.6471301288792514}}]
2002-10-24,88.360001,85.070000,34.029999,99.106295,0.673078,0.465888,0.607117,0.392883,0.0,NaN
2002-10-25,90.199997,85.379997,30.000000,100.489180,0.673078,0.465888,0.607117,0.392883,0.0,NaN
2002-10-28,89.610001,85.260002,31.070000,100.036162,0.673078,0.465888,0.607117,0.392883,0.0,NaN
2002-10-29,88.570000,86.370003,32.270000,99.853296,0.673078,0.465888,0.607117,0.392883,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2013-10-07,167.429993,106.139999,19.410000,201.114835,1.178912,0.035127,0.981534,0.018466,0.0,NaN
2013-10-08,165.479996,106.169998,20.340000,198.817015,1.178912,0.035127,0.981534,0.018466,0.0,NaN
2013-10-09,165.600006,105.320000,19.600000,198.928639,1.103492,0.152690,0.918462,0.081538,0.0,"[{'add_data': {'beta': -0.08877670441963324}, ..."


In [49]:
TLT_SPY2.output.describe()

,SPY,TLT,VIX,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs
count,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2761.0
mean,122.957346,96.676846,20.612252,143.172725,0.961823,0.242613,0.828027,0.171973,0.0
std,20.538104,12.043123,9.611184,25.760202,0.130203,0.145645,0.115827,0.115827,0.0
min,68.110001,80.650002,9.890000,92.349993,0.517940,0.000370,0.466025,0.000206,0.0
25%,110.202497,88.412502,14.280000,123.023692,0.900262,0.142692,0.766519,0.087643,0.0
50%,123.155003,92.115002,17.945001,139.980005,0.968337,0.238153,0.831126,0.168874,0.0
75%,137.047501,101.902502,23.885000,160.298494,1.029907,0.343428,0.912357,0.233481,0.0
max,173.050003,132.160004,80.860001,207.699934,1.242650,0.630784,0.999794,0.533975,0.0


In [50]:
TLT_SPY2.plot()

In [51]:
TLT_SPY2.stats

,Returns,Volatility,Sharpe
Results,0.098864,0.148952,0.663728


In [52]:
outperformance(TLT_SPY2)

Outperformance (Sharpe): 0.220


In [53]:
outperformance(TLT_SPY2, benchmark = TLT_SPY1, measure = "Volatility")

Outperformance (Volatility): -0.044


**Comments**
* The underperformance between 2007 and 2009 has returned, bringing pnl down below the inital investment amount
* Although annualised returns are much better (than buy-and-hold and 60/40) - it's interesting to note the significant increase in volatility vs the simple regression approach above 

## Using T-statistic instead of regression coefficients

Idea: Shifitng from regression coefficients to T-statistics should structurally increase my TLT allocation where the relationship is statistically stronger. I still think controlling for VIX is appropriate given the rational above. 

In [54]:
def TLT_SPY_3(tick, securities: List = ["SPY","TLT"], data = train_TSV, lookback = 180) -> Tuple[int, Dict]:
    index_loc_tic: int = data.index.get_loc(tick)
    lb_window = data.iloc[index_loc_tic-lookback:index_loc_tic]

    lb_log_ret = (np.log(lb_window)-np.log(lb_window.shift(1))).dropna(how = "any")

    reg_model = sm.regression.linear_model.OLS(lb_log_ret.SPY, lb_log_ret[["TLT","VIX"]])
    reg_results = reg_model.fit()

    TLT_coef: float = reg_results.tvalues.loc["TLT"]

    SPY_w = 1/(1+abs(TLT_coef))
    TLT_w = 1 - SPY_w

    sig: tuple = (SPY_w,TLT_w)

    return (sig, {"add_data":{"beta": TLT_coef}})

In [55]:
TLT_SPY3: Backtest = Backtest(name= "SPY TLT VIX dynamic weights + T statistics", data = train_TSV)
TLT_SPY3.securities = ["SPY", "TLT"]
TLT_SPY3.assign_rebal_attr(fixed_ticks=31)
TLT_SPY3.assign_signals(trading_signal= TLT_SPY_3)
TLT_SPY3.lookback_window = 60

In [56]:
TLT_SPY3.calculate()

,SPY,TLT,VIX,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs,Add_data
date,,,,,,,,,,
2002-10-23,90.199997,84.330002,33.200001,100.000000,0.271440,0.895484,0.244839,0.755161,NaN,[{'add_data': {'beta': -3.084322555846486}}]
2002-10-24,88.360001,85.070000,34.029999,100.163208,0.271440,0.895484,0.244839,0.755161,0.0,NaN
2002-10-25,90.199997,85.379997,30.000000,100.940254,0.271440,0.895484,0.244839,0.755161,0.0,NaN
2002-10-28,89.610001,85.260002,31.070000,100.672652,0.271440,0.895484,0.244839,0.755161,0.0,NaN
2002-10-29,88.570000,86.370003,32.270000,101.384341,0.271440,0.895484,0.244839,0.755161,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2013-10-07,167.429993,106.139999,19.410000,202.422185,0.905975,0.479018,0.749718,0.250282,0.0,NaN
2013-10-08,165.479996,106.169998,20.340000,200.669907,0.905975,0.479018,0.749718,0.250282,0.0,NaN
2013-10-09,165.600006,105.320000,19.600000,200.371469,0.502791,1.106415,0.414620,0.585380,0.0,"[{'add_data': {'beta': -1.4118450315039979}, '..."


In [57]:
TLT_SPY3.plot()

In [58]:
TLT_SPY3.stats

,Returns,Volatility,Sharpe
Results,0.097991,0.10574,0.926717


In [59]:
outperformance(TLT_SPY3)

Outperformance (Sharpe): 0.483


In [60]:
TLT_SPY3.output.describe()

,SPY,TLT,VIX,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs
count,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2762.000000,2761.0
mean,122.957346,96.676846,20.612252,146.385920,0.447726,0.915835,0.391050,0.608950,0.0
std,20.538104,12.043123,9.611184,36.464030,0.236360,0.374804,0.215471,0.215471,0.0
min,68.110001,80.650002,9.890000,100.000000,0.130056,0.004518,0.116754,0.002969,0.0
25%,110.202497,88.412502,14.280000,119.826836,0.285172,0.683529,0.232810,0.503300,0.0
50%,123.155003,92.115002,17.945001,133.278059,0.368613,0.925587,0.331204,0.668796,0.0
75%,137.047501,101.902502,23.885000,170.171382,0.492595,1.197899,0.496700,0.767190,0.0
max,173.050003,132.160004,80.860001,226.487050,1.225345,1.506269,0.997031,0.883246,0.0


In [61]:
outperformance(strategy= TLT_SPY3, benchmark=TLT_SPY0)

Outperformance (Sharpe): 0.227


**Comment**
* This approach manages the drawdown between 2007 and 2009 much better
* Slightly lower returns, but I expect this comes from the higher allocation to TLT

### Out of sample performance 

In [62]:
TLT_SPY3_OOS: Backtest = Backtest(name= "SPY TLT VIX dynamic weights + T statistics - Out of sample", data = TSV)
TLT_SPY3_OOS.securities = ["SPY", "TLT"]
TLT_SPY3_OOS.assign_rebal_attr(fixed_ticks=31)
TLT_SPY3_OOS.assign_signals(trading_signal= TLT_SPY_3)
TLT_SPY3_OOS.lookback_window = 60

In [63]:
TLT_SPY3_OOS.calculate()

,SPY,TLT,VIX,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs,Add_data
date,,,,,,,,,,
2002-10-23,90.199997,84.330002,33.200001,100.000000,0.271440,0.895484,0.244839,0.755161,NaN,[{'add_data': {'beta': -3.084322555846486}}]
2002-10-24,88.360001,85.070000,34.029999,100.163208,0.271440,0.895484,0.244839,0.755161,0.0,NaN
2002-10-25,90.199997,85.379997,30.000000,100.940254,0.271440,0.895484,0.244839,0.755161,0.0,NaN
2002-10-28,89.610001,85.260002,31.070000,100.672652,0.271440,0.895484,0.244839,0.755161,0.0,NaN
2002-10-29,88.570000,86.370003,32.270000,101.384341,0.271440,0.895484,0.244839,0.755161,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-12-24,601.299988,87.870003,14.270000,343.959453,0.529801,0.303974,0.920411,0.079589,0.0,NaN
2024-12-26,601.340027,87.820000,14.730000,343.965466,0.529801,0.303974,0.920411,0.079589,0.0,NaN
2024-12-27,595.010010,87.099998,15.950000,340.392955,0.529801,0.303974,0.920411,0.079589,0.0,NaN


In [101]:
TLT_SPY3_OOS.plot()

In [65]:
TLT_SPY3_OOS.stats

,Returns,Volatility,Sharpe
Results,0.082369,0.107901,0.763383


In [66]:
outperformance(TLT_SPY3_OOS)

Outperformance (Sharpe): 0.320


In [67]:
outperformance(TLT_SPY3_OOS, benchmark= TLT_SPY0)

Outperformance (Sharpe): 0.064


**Comment** 
* The Sharpe here is clearly less attractive out of sample which is cause for concern. 
* By virtue of being long only, this strategy will always underperform in periods of market distress as both stocks and bonds will go down

# Analysis - PCA

In [116]:
TSV_log_ret_demeaned = (TSV_log_ret - TSV_log_ret.mean())
PCA0 = sk.decomposition.PCA(n_components= 3)
PCA0.fit(TSV_log_ret_demeaned)
PCA0.set_output(transform = "pandas")
res0 = PCA0.transform(TSV_log_ret_demeaned)

In [117]:
explained_var0 = pd.DataFrame(PCA0.explained_variance_ratio_)
explained_var0.index = res0.columns
explained_var0.columns = ["Explained_Variance"]
explained_var0

,Explained_Variance
pca0,0.973652
pca1,0.016199
pca2,0.010149


In [118]:
eigen_vectors0 = pd.DataFrame(PCA0.components_)
eigen_vectors0.columns = res0.columns 
eigen_vectors0

,pca0,pca1,pca2
0,-0.119881,0.031893,0.992276
1,-0.562822,0.821171,-0.094390
2,0.817838,0.569791,0.080493


In [119]:
PCA_Factors0 = np.exp(np.cumsum(res0))
PCA_Factors0 = PCA_Factors0 / PCA_Factors0.iloc[0,:]
PCA_Factors0.index = TSV_log_ret_standard.index

In [120]:
fig6 = px.line(PCA_Factors0, x = PCA_Factors0.index, y = PCA_Factors0.columns)
fig6.show(renderer = "iframe")

In [121]:
adf_results_list: List = []
for x in range(len(res0.columns)):
    adf_results_list.append(sm.tsa.adfuller(res0.iloc[:,x]))

adf_results = pd.DataFrame(adf_results_list, index = res0.columns )
adf_results = adf_results[[0,2]]
adf_results.columns = ["T-statistic", "lag used"]
adf_results

,T-statistic,lag used
pca0,-29.841186,8
pca1,-26.073277,8
pca2,-77.891742,0


**Comment**
* Looking at the PCA factors it seems pretty that pca0 is simply capturing the variance (or volaility) from the VIX

In [123]:
PCA1 = sk.decomposition.PCA(n_components= 2)
PCA1.fit(TSV_log_ret_demeaned.iloc[:,:2])
PCA1.set_output(transform = "pandas")
res1 = PCA1.transform(TSV_log_ret_demeaned.iloc[:,:2])

In [124]:
explained_var1 = pd.DataFrame(PCA1.explained_variance_ratio_)
explained_var1.index = res1.columns
explained_var1.columns = ["Explained_Variance"]
explained_var1

,Explained_Variance
pca0,0.704377
pca1,0.295623


In [125]:
eigen_vectors1 = pd.DataFrame(PCA1.components_)
eigen_vectors1.columns = res1.columns 
eigen_vectors1

,pca0,pca1
0,0.904657,-0.426140
1,0.426140,0.904657


In [126]:
PCA_Factors1 = np.exp(np.cumsum(res1))
PCA_Factors1 = PCA_Factors1 / PCA_Factors1.iloc[0,:]
PCA_Factors1.index = TSV_log_ret_demeaned.index

In [127]:
fig7 = px.line(PCA_Factors1, x = PCA_Factors1.index, y = PCA_Factors1.columns)
fig7.show(renderer = "iframe")

In [128]:
adf_results_list: List = []
for x in range(len(res1.columns)):
    adf_results_list.append(sm.tsa.adfuller(res1.iloc[:,x]))


adf_results = pd.DataFrame(adf_results_list, index = res1.columns )
adf_results = adf_results[[0,2]]
adf_results.columns = ["T-statistic", "lag used"]
adf_results

,T-statistic,lag used
pca0,-19.444369,15
pca1,-55.774543,1


# Long short strategy - PCA mean reversion + T-statistic weightings 

In [129]:
TSVF0 = TSV.merge(PCA_Factors0, left_index= True, right_index = True)
TSVF1 = TSV.merge(PCA_Factors1, left_index= True, right_index = True)

In [130]:
train_TSVF0 = TSVF0.iloc[:int(len(TSVF0)/2)]
train_TSVF1 = TSVF0.iloc[:int(len(TSVF0)/2)]

In [133]:
def TLT_SPY_4(tick, securities: List = ["SPY","TLT"], data = train_TSVF0, lookback = 180) -> Tuple[int, Dict]:
    index_loc_tic: int = data.index.get_loc(tick)
    lb_window = data.iloc[index_loc_tic-lookback:index_loc_tic]
    
    lb_window1 = lb_window[securities + ["VIX"]]
    rolling_mean = lb_window.mean()

    lb_log_ret = (np.log(lb_window1)-np.log(lb_window1.shift(1))).dropna(how = "any")
    
    reg_model = sm.regression.linear_model.OLS(lb_log_ret.SPY, lb_log_ret[["TLT","VIX"]])
    reg_results = reg_model.fit()

    TLT_coef: float = reg_results.tvalues.loc["TLT"]

    SPY_w = 1/(1+abs(TLT_coef))
    TLT_w = 1 - SPY_w
    
    
    if lb_window.pca2.iloc[-1] >= rolling_mean.pca2:
        sig: tuple = (-SPY_w,-TLT_w)
    elif lb_window.pca2.iloc[-1] < rolling_mean.pca2:
        sig: tuple = (SPY_w,TLT_w)
    else:
        sig = (0,0)


    return (sig, {"add_data":{"rolling_mean": rolling_mean}})

In [134]:
TLT_SPY4: Backtest = Backtest(name= "PCA long-short - mean-reversion", data = train_TSVF0)
TLT_SPY4.securities = ["SPY", "TLT"]
TLT_SPY4.assign_rebal_attr(fixed_ticks=31)
TLT_SPY4.assign_signals(trading_signal= TLT_SPY_4)
TLT_SPY4.lookback_window = 60

In [135]:
TLT_SPY4.calculate()

,SPY,TLT,VIX,pca0,pca1,pca2,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs,Add_data
date,,,,,,,,,,,,,
2002-10-24,88.360001,85.070000,34.029999,1.076366,1.047775,0.980726,100.000000,0.264223,0.901061,0.233468,0.766532,NaN,[{'add_data': {'rolling_mean': SPY 88.2131...
2002-10-25,90.199997,85.379997,30.000000,0.947726,1.051396,0.989112,100.765496,0.264223,0.901061,0.233468,0.766532,0.0,NaN
2002-10-28,89.610001,85.260002,31.070000,0.982135,1.050761,0.985542,100.501483,0.264223,0.901061,0.233468,0.766532,0.0,NaN
2002-10-29,88.570000,86.370003,32.270000,1.021768,1.065342,0.986139,101.226868,0.264223,0.901061,0.233468,0.766532,0.0,NaN
2002-10-30,89.430000,86.339996,31.230000,0.988077,1.062718,0.990883,101.427063,0.264223,0.901061,0.233468,0.766532,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-10-08,165.479996,106.169998,20.340000,0.903301,1.463159,0.851235,190.544026,0.860338,0.454889,0.749718,0.250282,0.0,NaN
2013-10-09,165.600006,105.320000,19.600000,0.870515,1.458272,0.845096,190.260621,-0.477420,-1.050585,-0.414620,-0.585380,0.0,[{'add_data': {'rolling_mean': SPY 168.175...
2013-10-10,169.169998,105.489998,16.480000,0.731197,1.466830,0.848600,188.377639,-0.477420,-1.050585,-0.414620,-0.585380,0.0,NaN


In [136]:
TLT_SPY4.plot()

In [137]:
TLT_SPY4.stats

,Returns,Volatility,Sharpe
Results,0.08735,0.105777,0.825798


In [138]:
outperformance(TLT_SPY4)

Outperformance (Sharpe): 0.382


# Long short strategy - PCA mean reversion (bands)

Calculate confidence interval and trade if PCA is outside of bollinger band - towards mean 

In [139]:
def TLT_SPY_5(tick, securities: List = ["SPY","TLT"], data = train_TSVF0, lookback = 180) -> Tuple[int, Dict]:
    index_loc_tic: int = data.index.get_loc(tick)
    lb_window = data.iloc[index_loc_tic-lookback:index_loc_tic]
    
    lb_window1 = lb_window[securities + ["VIX"]]
    rolling_mean = lb_window.mean()

    lb_log_ret = (np.log(lb_window1)-np.log(lb_window1.shift(1))).dropna(how = "any")

    reg_model = sm.regression.linear_model.OLS(lb_log_ret.SPY, lb_log_ret[["TLT","VIX"]])
    reg_results = reg_model.fit()

    TLT_coef: float = reg_results.tvalues.loc["TLT"]

    SPY_w = 1/(1+abs(TLT_coef))
    TLT_w = 1 - SPY_w

    k = sci.stats.t.ppf(0.975, lookback-1)
    

    upper_bound =  rolling_mean.pca2 + (k * lb_window.pca2.std() / np.sqrt(lookback-1))
    lower_bound = rolling_mean.pca2 - (k * (lb_window.pca2.std() / np.sqrt(lookback-1)))  
    
    
    if lb_window.pca2.iloc[-1]  >= upper_bound:
        sig: tuple = (-SPY_w,-TLT_w)
    elif lb_window.pca2.iloc[-1] < lower_bound:
        sig: tuple = (SPY_w,TLT_w)
    else:
        sig = (0,0)


    return (sig, {"add_data":{"rolling_mean": rolling_mean}})

In [140]:
TLT_SPY5: Backtest = Backtest(name= "PCA long-short - Mean-reversion", data = train_TSVF0)
TLT_SPY5.securities = ["SPY", "TLT"]
TLT_SPY5.assign_rebal_attr(fixed_ticks=31)
TLT_SPY5.assign_signals(trading_signal= TLT_SPY_5)
TLT_SPY5.lookback_window = 60

In [141]:
TLT_SPY5.calculate()

,SPY,TLT,VIX,pca0,pca1,pca2,PNL,SPY holdings,TLT holdings,SPY weights,TLT weights,Costs,Add_data
date,,,,,,,,,,,,,
2002-10-24,88.360001,85.070000,34.029999,1.076366,1.047775,0.980726,100.000000,0.264223,0.901061,0.233468,0.766532,NaN,[{'add_data': {'rolling_mean': SPY 88.2131...
2002-10-25,90.199997,85.379997,30.000000,0.947726,1.051396,0.989112,100.765496,0.264223,0.901061,0.233468,0.766532,0.0,NaN
2002-10-28,89.610001,85.260002,31.070000,0.982135,1.050761,0.985542,100.501483,0.264223,0.901061,0.233468,0.766532,0.0,NaN
2002-10-29,88.570000,86.370003,32.270000,1.021768,1.065342,0.986139,101.226868,0.264223,0.901061,0.233468,0.766532,0.0,NaN
2002-10-30,89.430000,86.339996,31.230000,0.988077,1.062718,0.990883,101.427063,0.264223,0.901061,0.233468,0.766532,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-10-08,165.479996,106.169998,20.340000,0.903301,1.463159,0.851235,186.376738,0.841522,0.444940,0.749718,0.250282,0.0,NaN
2013-10-09,165.600006,105.320000,19.600000,0.870515,1.458272,0.845096,186.099531,-0.466978,-1.027608,-0.414620,-0.585380,0.0,[{'add_data': {'rolling_mean': SPY 168.175...
2013-10-10,169.169998,105.489998,16.480000,0.731197,1.466830,0.848600,184.257730,-0.466978,-1.027608,-0.414620,-0.585380,0.0,NaN


In [142]:
TLT_SPY5.plot()

In [143]:
TLT_SPY5.stats

,Returns,Volatility,Sharpe
Results,0.084177,0.103212,0.815581


In [144]:
outperformance(TLT_SPY5)

Outperformance (Sharpe): 0.372


**Next steps**

* Look at PNL attributions between the two assets to get a better idea of when each strategy performs well
* The relationship between bonds and equities is attenuated by many other factors (e.g Inflation, interest rates, Economic growth), so looking at Macro data or approaches to identify regimes (Hidden Markov Models) may be interesting
* Want investigate pairs trading between these two (co-integration tests)
* Look at properly evaluating the role of volatility (both change and level)
  